#### load dependencies

In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import LSTM
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import os
from keras.layers import Conv1D,MaxPooling1D
from keras.datasets import imdb
%matplotlib inline

### set hyperparameters

In [ ]:
output_dir = "model_output/CNN_BI_LSTM_Stacked/"

epochs = 4
batch_size = 128

#embedding layer:
n_dims = 64
#increasing review length as lstm now works in both ways

max_review_length = 200
n_unique_words = 10000
#while padding if the review is < 100 words pad it at the beginning of the review.
#while truncating if the review is > 100 cut the review at the beginning
pad_type = trunc_type = "pre"
dropout_embed = 0.2
#conv layer:
n_conv = 64
k_conv = 3
max_pool = 4

n_lstm = 64
dropout_lstm = 0.2


#dense_layer:
#n_dense = 256
#dropout = 0.2

### load data

In [ ]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=n_unique_words)

####  preprocessing data

In [ ]:
x_train = pad_sequences(x_train,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)
x_test = pad_sequences(x_test,maxlen=max_review_length,padding=pad_type,truncating=trunc_type,value=0)

#### model architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words,n_dims,input_length=max_review_length))
model.add(SpatialDropout1D(dropout_embed))
model.add(Conv1D(n_conv,k_conv,activation="relu"))
model.add(MaxPooling1D(max_pool))
model.add(Bidirectional(LSTM(units=n_lstm,dropout=dropout_lstm)))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])

### save the model weights for every epoch

In [ ]:
modelcheckpoint  = ModelCheckpoint(filepath=output_dir +"weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### train the model

In [ ]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test),verbose=1, callbacks=[modelcheckpoint])

### load the epoch with highest validation accuracy

In [ ]:
model.load_weights(filepath=output_dir+"/weights.01.hdf5")

### make predictions(probability) on test set

In [ ]:
y_pred = model.predict_proba(x_test)

In [ ]:
y_pred[0]

In [ ]:
plt.hist(y_pred)
_ = plt.axvline(x = 0.5, color="red")

In [ ]:
auc = roc_auc_score(y_test,y_pred)*100.0

In [ ]:
auc

In [ ]:
predicted_sentiment = []
for i in y_pred:
    predicted_sentiment.append(i[0])

In [ ]:
df = pd.DataFrame(list(zip(predicted_sentiment,y_test)),columns=["predicted","real"])

In [ ]:
df.head()